In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

melb_file_path = 'assets/input/melb_data.csv'
melbourne_data = pd.read_csv(melb_file_path)
prep_melb_data = melbourne_data.dropna(axis=0)

y = prep_melb_data.Price
features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X = prep_melb_data[features]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

# Overfitting y Underfitting
Ahora que tenemos una herramienta para medir la precisión de nuestro modelo, podríamos utilizarla para evaluar la calidad de varios modelos y luego quedarnos con el que mejor performance muestre.

Recordamos, también, que **una de las características de un modelo es su profundidad** (cantidad de *splits*, divisiones en grupos). Cada nivel que agregamos, nos divide el grupo inicial en 2, por tanto, si, por ejemplo, tuviésemos 10 niveles, esto nos daría un total de 1024 grupos de elementos, donde **cada grupo, a medida que crece la profundidad, contendrá una menor cantidad de elementos**. Esto generará que nuestro modelo se ajuste con gran precisión a cada uno de esos elementos (de entrenamiento) pero como contrapartida, **si se ajusta tanto a los elementos particulares perderá precisión al presentarse nuevos elementos**. Esto es lo que se conoce como **overfitting**.

Por otro lado, si utilizáramos un **bajo nivel de profundidad, diviendo nuestros datos en 2 o 4 grupos** por ejemplo, nuestro modelo **no lograría capturar distinciones o patrones importantes**, además que cada grupo contendría una dispersión importante entre sus valores. Esto es lo que se conoce como **underffiting**


*foto no esta cargando*

Pasando a la práctica, una forma de controlar la profundidad de nuestro modelo, es mediante *max_leaf_nodes*. Podremos, entonces, medir la precisión de diferentes modelos según su profundidad y luego quedarnos con el que presente un mejor resultado.

In [13]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaves, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaves, random_state=1)
    model.fit(train_X, train_y)
    predictions = model.predict(val_X)
    mae = mean_absolute_error(val_y, predictions)
    return(mae)

candidate_leaves = [5, 50, 500, 5000]
maes = {leaves: get_mae(leaves,train_X, val_X, train_y, val_y) for leaves in candidate_leaves}

# Modelo con mejor performance (menor MAE)
# esta funcion para encontrar el minimo la saque de internet
best_tree_size = min(maes, key=maes.get)
print('El modelo de mejor MAE fue el que tenía {} leaves, con un error de: {:,.2f}'.format(best_tree_size, maes[best_tree_size]))


El modelo de mejor MAE fue el que tenía 500 leaves, con un error de: 260,235.05
Max leaves: 5    		 Mean Error: 385696.54278937966
Max leaves: 50    		 Mean Error: 279794.61143891385
Max leaves: 500    		 Mean Error: 260235.05178963946
Max leaves: 5000    		 Mean Error: 273765.3918657198


**IMPORTANTE**: luego de encontrar el **mejor nivel de profundidad** para nuestro modelo, se **construye el modelo final, incluyendo tanto la data de entrenamiento como la de validación**, ya que esto nos permitirá obtener la **mayor precisión** posible.

In [14]:
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)
final_model.fit(X, y)

DecisionTreeRegressor(max_leaf_nodes=500, random_state=1)